In [1]:
import numpy as np
from astropy.table import Table

In [2]:
raw = Table.read('./data/cos_NFPMM_newspec.fits', format='fits')


In [3]:
bflags = ['b0.5', 'b1', 'b2', 'b3']

In [4]:
bflag_weak = bflags[2]
bflag_strong = bflags[1]

In [17]:
bands = 'uBVrizYJH'
bands = list(bands)
flux_flag = 'FLUX_RADIUS'

In [18]:
bands[4] = 'ip'
bands[5] = 'zpp'

In [19]:
raw['RADIUSERR'] = 0.2 * np.std(raw['FLUX_RADIUS'])
raw = raw[ (raw['FLAG_HST']==1) & (raw['pot_SOM']==1) ]        ## pot_SOM: type=0, ip_MAG_AUTO<24.5, ZPDF>0

mask_pure =  (raw['ID']>0) & (raw['b2']==0)                    ## primary match and no additional match
pure = raw[mask_pure]

mask_bl = (raw['ID']>0) & (raw[bflag_weak]>0)                  ## primary match and at least one additional
weak_unrec = raw[mask_bl]

mask_bl = (raw['ID']>0) & (raw[bflag_strong]>0)
strong_unrec = raw[mask_bl]

In [20]:
mask_bl = (raw['ID']>0) & (raw[bflag_weak]>0) & (raw[bflag_strong]==0)              ## primary match and at least one additional
true_weak_unrec = raw[mask_bl]

In [21]:
for k in weak_unrec['NUMBER']:
    if k in true_weak_unrec['NUMBER'] or k in strong_unrec['NUMBER']:
        continue
    else:
        print(f"ISSUE WITH {k}")

In [22]:
for k in true_weak_unrec['NUMBER']:
    if k in weak_unrec['NUMBER']:
        continue
    else:
        print(f"ISSUE WITH {k}")

In [23]:
for k in strong_unrec['NUMBER']:
    if k in true_weak_unrec['NUMBER']:
        print(f"ISSUE WITH {k}")        

In [24]:
def get_table(sample, bscore):
    new_table = np.zeros((len(sample),len(bands)*2 + 2))
    
    for i,b in enumerate(bands):
        new_table[:,2*i] = sample[b]
        new_table[:,2*i+1] = sample[b+'ERR']
    new_table[:,-2] = sample[flux_flag]
    new_table[:,-1] = bscore
    
    return new_table

In [25]:
pure_table = get_table(pure, 0)
weak_table = get_table(true_weak_unrec, 1)
strong_table = get_table(strong_unrec, 2)

In [26]:
pure_table[0]

array([2.36389084e+01, 4.99182157e-02, 2.26036606e+01, 2.07192078e-02,
       2.10741158e+01, 1.32745933e-02, 2.03422451e+01, 7.61330267e-03,
       1.97589417e+01, 4.60653659e-03, 1.92758007e+01, 1.48872042e-03,
       1.91422443e+01, 1.64770265e-03, 1.88291759e+01, 1.80094677e-03,
       1.85646744e+01, 2.22367607e-03, 6.47549248e+00, 0.00000000e+00])

In [27]:
csv_header =  ['u', 'uERR', 'B', 'BERR', 'V', 'VERR', 'r', 'rERR',
               'ip', 'ipERR', 'zpp', 'zppERR', 'Y', 'YERR', 'J', 'JERR',
               'H', 'HERR', 'FLUX_RADIUS', 'blend']
csv_headstr = ','.join(csv_header)

In [28]:
np.savetxt('./data/new_run/pure_table_err.csv', pure_table, delimiter=',', header=csv_headstr, comments='')
np.savetxt('./data/new_run/weak_table_err.csv', weak_table, delimiter=',', header=csv_headstr, comments='')
np.savetxt('./data/new_run/strong_table_err.csv', strong_table, delimiter=',', header=csv_headstr, comments='')